# MNIST画像の分類 - Part1:モデルのトレーニング

MNISTデータセットに対し、scikit-learnを使用した画像の分類モデルを作成します。ここでは、トレーニング環境としてAzure上のリモートクラスタを使用して分類モデルの作成及びトレーニングを行います。
<br><br>
こちらは、以下のチュートリアルをベースにしています。
<br>
https://docs.microsoft.com/ja-jp/azure/machine-learning/service/tutorial-train-models-with-aml
<br>
<br>
注意：utils.pyが必要になりますので、GitHubからクローンしない場合は手動で作成してください。

## 1．開発環境の設定

必要なパッケージをインポートします。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import azureml.core
from azureml.core import Workspace

ワークブックに接続します。

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep = '\t')

実験を作成します。

In [ ]:
experiment_name = "skleran-mnist"

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

初期設定で作成した、コンピューティングリソースであるMachine Learning Computeの仮想マシンクラスタをアタッチします。

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# CPUクラスタの場合
compute_name = "cpucluster"
# GPUクラスタの場合
# compute_name = "gpucluster"

# クラスタへのアタッチもしくはクラスタの作成
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    
    if compute_target and type(compute_target) is AmlCompute:
        print("Found compute target: " + compute_name)
else:
    print("Please create a new compute target.")

## 2．データの調査

MNITSデータセットをダウンロードし、dataディレクトリに保存します。ここでは、トレーニング用とテスト用の両方のイメージとラベルをダウンロードします。

In [ ]:
import urllib.request
import os

data_folder = os.path.join(os.getcwd(), "data")
os.makedirs(data_folder, exist_ok=True)

urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', filename=os.path.join(data_folder, 'train-images.gz'))
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', filename=os.path.join(data_folder, 'train-labels.gz'))
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', filename=os.path.join(data_folder, 'test-images.gz'))
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', filename=os.path.join(data_folder, 'test-labels.gz'))

MNISTデータセットを、トレーニングデータとテストデータに分割します。

In [ ]:
from utils import load_data

X_train = load_data(os.path.join(data_folder, 'train-images.gz'), False) / 255.0
X_test = load_data(os.path.join(data_folder, 'test-images.gz'), False) / 255.0
y_train = load_data(os.path.join(data_folder, 'train-labels.gz'), True).reshape(-1)
y_test = load_data(os.path.join(data_folder, 'test-labels.gz'), True).reshape(-1)

いくつかのイメージを、サンプルとして表示します。

In [ ]:
count = 0
sample_size = 30
plt.figure(figsize=(16,6))

for i in np.random.permutation(X_train.shape[0])[:sample_size]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x=10, y=-10, s=y_train[i], fontsize=18)
    plt.imshow(X_train[i].reshape(28, 28), cmap=plt.cm.Greys)
plt.show()

トレーニングデータおよびテストデータを、Azureにアップロードします。

In [ ]:
ds = ws.get_default_datastore()
print("Default datastore: ", ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir=data_folder, target_path="mnist", overwrite=True, show_progress=True)

## 3．リモートクラスタでのトレーニング

リモートクラスタに送信するスクリプトを格納するディレクトリを作成します。

In [ ]:
import os

script_folder = os.path.join(os.getcwd(), "sklearn-mnist")
os.makedirs(script_folder, exist_ok=True)

utils.pyを送信するディレクトリにコピーします。

In [ ]:
import shutil

shutil.copy("utils.py", script_folder)

リモートクラスタ上で実行するトレーニングスクリプトを作成します。

In [ ]:
%%writefile $script_folder/train.py

import argparse
import os
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

from azureml.core import Run
from utils import load_data

# 外部パラメータを指定
parser = argparse.ArgumentParser()
parser.add_argument("--data-folder", type=str, dest="data_folder", help="data folder mounting point")
parser.add_argument("--regularization", type=float, dest="reg", default=0.01, help="regularization rate")
args = parser.parse_args()

data_folder = args.data_folder
print("Data folder:", data_folder)

# トレーニングデータとテストデータの取得 -- データは0-1に正規化
X_train = load_data(os.path.join(data_folder, 'train-images.gz'), False) / 255.0
X_test = load_data(os.path.join(data_folder, 'test-images.gz'), False) / 255.0
y_train = load_data(os.path.join(data_folder, 'train-labels.gz'), True).reshape(-1)
y_test = load_data(os.path.join(data_folder, 'test-labels.gz'), True).reshape(-1)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep = '\n')

# トレーニングの実行
run = Run.get_context()

print("Train a logistic regression model with regularization rate of", args.reg)
clf = LogisticRegression(C=1.0/args.reg, solver="liblinear", multi_class="auto", random_state=42)
clf.fit(X_train, y_train)

# 予測精度の計算
print("Tredict the test set")
y_hat = clf.predict(X_test)

acc = np.average(y_hat==y_test)
print("Accuracy is", acc)

run.log("regularization rate", np.float(args.reg))
run.log("accuracy", np.float(acc))

# モデルファイルの保存
os.makedirs("outputs", exist_ok=True)
joblib.dump(value=clf, filename="outputs/sklearn_mnist_model.pkl")

SKLearn Estimatorを作成し、トレーニングの実行処理を送信します。送信処理は非同期になります。

In [ ]:
# SKLearn Estimatorの作成
from azureml.train.sklearn import SKLearn

script_params = {
    "--data-folder": ds.path("mnist").as_mount(),
    "--regularization": 0.5
}

est = SKLearn(source_directory=script_folder,
             script_params=script_params,
             compute_target=compute_target,
             entry_script="train.py")

# クラスターへのジョブの送信
run = exp.submit(config=est)
run

## 4．トレーニング処理の監視

Jupyter Widgetを使用して、実行状況を監視します。

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

完了後にログの結果を取得します。

In [ ]:
run.wait_for_completion(show_output=False)

実行結果を取得します。

In [ ]:
print(run.get_metrics())

## 5．モデルの登録

トレーニングの実行で出力されたファイルを確認します。ファイルはAzure Portalからも確認できます。

In [ ]:
print(run.get_file_names())

ワークスペースにモデルを登録します。これにより、ワークスペースにてモデルが共有されます。

In [ ]:
model = run.register_model(model_name="sklearn_mnist", model_path="outputs/sklearn_mnist_model.pkl")
print(model.name, model.id, model.version, sep="\t")